In [101]:
import torch
import torch.nn as nn
import cv2
import torchvision
from torch.utils.data import DataLoader
from PIL import Image
from pathlib import Path
from torchvision import transforms
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [102]:
from model import CarDetection

carDetectionModel = CarDetection(3, 16, 1, 0.3)
carDetectionModel.load_state_dict(torch.load('car-detection-model'))

<All keys matched successfully>

In [103]:
def predict(model, img):
    model.eval()
    with torch.inference_mode():
        y = model(img).squeeze()
        y = torch.sigmoid(y)
    return y

In [104]:
CLASSES = ["Non-Vehicles", "Vehicles"]
path = Path('./data/test/non-vehicles/5.png')

img = Image.open(path)
tensor_transform = transforms.ToTensor()
img = tensor_transform(img).unsqueeze(0)

In [107]:
print(CLASSES[int(torch.round(predict(carDetectionModel, img)))])

Non-Vehicles


Expected Image shape: [1, 3, 64, 64] <br>
We can iterate over the image, 

In [112]:
carPath = Path('./data/number-plate-data/car-parked.jpg')
carImg = Image.open(carPath)

tensor_transform = transforms.Compose([
    transforms.Resize((360, 480)),
    transforms.ToTensor()
])
carImg = tensor_transform(carImg)
stride = 30
rShape = 64
cShape = 64
cols = carImg.shape[2]
rows = carImg.shape[1]

start_time = timer()
for r in range(0, rows-rShape, stride):
    for c in range(0, cols-cShape, stride):
        updatedImg = carImg[:, r:r+rShape, c:c+cShape]
        prediction = predict(carDetectionModel, updatedImg.unsqueeze(0))
        # if(prediction > 0.94):
        #     plt.imshow(updatedImg.permute(1,2,0).numpy())
        #     plt.show()
        #     print(prediction)
end_time = timer()
print(end_time-start_time)

0.23330390000046464
